In [36]:
import gradio as gr
import sys
import io
import os
import re
import subprocess
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from google import genai


load_dotenv()
open_ai_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
print(open_ai_key[:8])
print(anthropic_api_key[:8])
print(deepseek_api_key[:8])
print(google_api_key[:8])


openai_client = OpenAI()
anthropic_client = Anthropic()
deepseek_client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")
google_client = genai.Client()

css = """
.python {
    background-color: #b0ffc5;
}
.cpp {
    background-color: #ffeeb0;
}
.java {
    background-color: #f8dcfc;
}
"""

cpp_code="""#include <iostream>
#include <vector>
#include <algorithm>
#include <chrono>

class SortMatrixByDiagonals {
public:
    std::vector<std::vector<int>> sortMatrix(std::vector<std::vector<int>>& grid) {
        int n = grid.size();

        for (int i = 1; i < n; i++) {
            std::vector<int> arr(n - i);
            for (int j = i; j < n; j++) {
                arr[j - i] = grid[j - i][j];
            }
            std::sort(arr.begin(), arr.end());
            for (int j = i; j < n; j++) {
                grid[j - i][j] = arr[j - i];
            }
        }

        for (int i = 0; i < n; i++) {
            std::vector<int> arr(n - i);
            for (int j = i; j < n; j++) {
                arr[j - i] = grid[j][j - i];
            }
            std::sort(arr.begin(), arr.end());
            for (int j = i; j < n; j++) {
                grid[j][j - i] = arr[n - j - 1];
            }
        }

        return grid;
    }
};

int main() {
    SortMatrixByDiagonals sol;
    std::vector<std::vector<int>> grid = {
        {1, 7, 3},
        {9, 8, 2},
        {4, 5, 6}
    };
    auto start = std::chrono::high_resolution_clock::now();
    auto res = sol.sortMatrix(grid);
    auto duration = std::chrono::high_resolution_clock::now() - start;
    std::cout << "Solved in " << std::chrono::duration_cast<std::chrono::microseconds>(duration).count() / 1000.0 << " millis\n";
    
    for (const auto& arr : res) {
        std::cout << "[";
        for (size_t i = 0; i < arr.size(); ++i) {
            std::cout << arr[i];
            if (i < arr.size() - 1) std::cout << ", ";
        }
        std::cout << "]\n";
    }
    return 0;
}
"""

python_code = """class SortMatrixByDiagonals:
    def sort_matrix(self, grid):
        n = len(grid)

        for i in range(1, n):
            arr = [grid[j-i][j] for j in range(i, n)]
            arr.sort()
            for j in range(i, n):
                grid[j-i][j] = arr[j-i]

        for i in range(n):
            arr = [grid[j][j-i] for j in range(i, n)]
            arr.sort()
            for j in range(i, n):
                grid[j][j-i] = arr[n-j-1]

        return grid


if __name__ == "__main__":
    sol = SortMatrixByDiagonals()
    grid = [
        [1, 7, 3],
        [9, 8, 2],
        [4, 5, 6]
    ]
    
    import time
    start = time.time()
    res = sol.sort_matrix(grid)
    duration = (time.time() - start) * 1000
    print(f"Solved in {duration:.3f} millis")
    for arr in res:
        print(arr)
"""

java_code = """import java.util.Arrays;

public class SortMatrixByDiagonals {
    public int[][] sortMatrix(int[][] grid) {
        int n = grid.length;

        for (int i = 1; i < n; i++) {
            int[] arr = new int[n-i];
            for (int j = i; j < n; j++) {
                arr[j-i] = grid[j-i][j];
            }
            Arrays.sort(arr);
            for (int j = i; j < n; j++) {
                grid[j-i][j] = arr[j-i];
            }
        }

        for (int i = 0; i < n; i++) {
            int[] arr = new int[n-i];
            for (int j = i; j < n; j++) {
                arr[j-i] = grid[j][j-i];
            }
            Arrays.sort(arr);
            for (int j = i; j < n; j++) {
                grid[j][j-i] = arr[n-j-1];
            }
        }

        return grid;
    }

    public static void main(String[] args) {
        var sol = new SortMatrixByDiagonals();
        int[][] grid;
        int[][] res;

        grid = new int[][] {
                new int[] {1, 7, 3},
                new int[] {9, 8, 2},
                new int[] {4, 5, 6}
        };
        long start = System.nanoTime();
        res = sol.sortMatrix(grid);
        long duration = System.nanoTime() - start;
        System.out.printf("Solved in %.3f millis.\\n", duration / 1000_000.0);
        for (int[] arr: res) {
            System.out.println(Arrays.toString(arr));
        }
    }
}
"""


def stream_gpt(code, sys_prompt):
    messages = [
        {'role': 'system', 'content': sys_prompt},
        {'role': 'user', 'content': code}
    ]

    stream = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True,
    )

    resp = ''
    for chunk in stream:
        text = chunk.choices[0].delta.content or ''
        resp += text
        yield resp


def stream_claude(code, sys_prompt):
    resp = ''
    with anthropic_client.messages.stream(
        max_tokens=1000,
        system=sys_prompt,
        messages=[{"role": "user", "content": code}],
        model="claude-sonnet-4-20250514",
    ) as stream:
        for text in stream.text_stream:
            resp += text
            yield resp


def stream_deepseek(code, sys_prompt):
    messages = [
        {'role': 'system', 'content': sys_prompt},
        {'role': 'user', 'content': code}
    ]

    stream = deepseek_client.chat.completions.create(
        model='deepseek-chat',
        messages=messages,
        stream=True,
    )

    resp = ''
    for chunk in stream:
        text = chunk.choices[0].delta.content or ''
        resp += text
        yield resp


def stream_gemini(code, sys_prompt):
    stream = google_client.models.generate_content_stream(
        model="gemini-2.5-flash",
        config=genai.types.GenerateContentConfig(
            system_instruction=sys_prompt
        ),
        contents=[code]
    )

    resp = ''
    for chunk in stream:
        resp += chunk.text
        yield resp


def run_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
        out = output.getvalue()
    except Exception as e:
        sys.stdout = sys.__stdout__
        out = e
    return out

    
def run_java(code):
    match = re.search(r"\bclass\s+([A-Za-z_][A-Za-z0-9_]*)", java_code)
    if match:
        class_name = match.group(1)
        print("Class name:", class_name)
    
        with open(f"{class_name}.java", "w") as f:
            f.write(java_code)

        try:
            subprocess.run(["javac", f"{class_name}.java"], check=True)
            result = subprocess.run(["java", class_name], check=True, text=True, capture_output=True)
            return result.stdout
        except subprocess.CalledProcessError as e:
            return e.stderr
    else:
        return 'Failed to run java. Class name not found.'
        

def run_cpp(code):
    with open("code.cpp", "w") as f:
        f.write(code)
    try:
        subprocess.run(["clang++", "-std=c++17", "-o", "code", "code.cpp"], check=True)
        result = subprocess.run(["./code"], check=True, text=True, capture_output=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return e.stderr


In [37]:
def convert(model, code, sys_prompt):
    if model == 'gpt':
        stream = stream_gpt(code, sys_prompt)
    elif model == 'Claude':
        stream = stream_claude(code, sys_prompt)
    elif model == 'Deepseek':
        stream = stream_deepseek(code, sys_prompt)
    elif model == 'Gemini':
        stream = stream_gemini(code, sys_prompt)
    else :
        stream = [f"Here will be converted by {model} c++ code."]

    for chunk in stream:
        yield chunk


def convert_python_to_cpp(model, python):
    sys_prompt = ("You are code converter expert. You convert python into optimized c++ code."
        "Please do all printings to stdout." 
        "Respond only with c++ code ready to be executed. Don't wrap the code with cpp``` or any of this.")
    stream = convert(model, python, sys_prompt)
    for chunk in stream:
        yield chunk


def convert_java_to_cpp(model, java):
    sys_prompt = ("You are code converter expert. You convert java into optimized c++ code."
        "Please do all printings to stdout." 
        "Respond only with c++ code ready to be executed. Don't wrap the code with cpp``` or any of this.")
    stream = convert(model, java, sys_prompt)
    for chunk in stream:
        yield chunk


def convert_java_to_python(model, java):
    sys_prompt = ('You are code converter expert. You convert java code into analogous python code.' 
            "Respond only with python code ready to be executed. Don't wrap the code with python``` or any of this.")
    stream = convert(model, java, sys_prompt)
    for chunk in stream:
        yield chunk


with gr.Blocks(css=css) as ui:
    with gr.Row():
        python = gr.TextArea(label="Python code", value=python_code)
        java = gr.TextArea(label="Java code", value=java_code)
        cpp = gr.TextArea(label="C++ code", value=cpp_code)
    with gr.Row():
        model = gr.Dropdown(["gpt", "Claude", "Deepseek", "Gemini"], value="gpt", label="Model")
    with gr.Row():
        convert_python_to_cpp_btn = gr.Button(value="Convert python to C++")
        convert_java_to_cpp_btn = gr.Button(value="Convert java to C++")
        convert_java_to_python_btn = gr.Button(value="Convert java to python")
    with gr.Row():
        run_python_btn = gr.Button(value="Run Python")
        run_java_btn = gr.Button(value="Run Java")
        run_cpp_btn = gr.Button(value="Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python output", lines=3, elem_classes="python")
        java_out = gr.TextArea(label="Java output", lines=3, elem_classes="java")
        cpp_out = gr.TextArea(label="C++ output", lines=3, elem_classes="cpp")

    convert_python_to_cpp_btn.click(convert_python_to_cpp, [model, python], cpp)
    convert_java_to_cpp_btn.click(convert_java_to_cpp, [model, java], cpp)
    convert_java_to_python_btn.click(convert_java_to_python, [model, java], python)
    run_python_btn.click(run_python, python, python_out)
    run_java_btn.click(run_java, java, java_out)
    run_cpp_btn.click(run_cpp, cpp, cpp_out)

ui.launch(inbrowser=True)